In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("rotten_tomatoes")

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
dataset

NameError: name 'dataset' is not defined

In [ ]:
dataset["test"][0] ##üc satır var train test validation ve iki sutun var text label

{'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .',
 'label': 1}

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased"
)

In [ ]:
tokenizer(dataset["train"][0]["text"]) # train kısmındaki ilk satırın text kısmını al demek

{'input_ids': [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation = True)

In [ ]:
dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,
                                       return_tensors = "tf")

In [ ]:
from transformers import TFAutoModelForSequenceClassification
### transformer kütüphanesi pytorch ile yazılmıştır ve eger tensorflow ile kullanmak istersen basina TF getir

In [ ]:
my_model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased"
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tf_train_set = my_model.prepare_tf_dataset(
    dataset["train"],
    shuffle = True, #Bu parametre, veri kümesinin her epoch'ta karıştırılacağını belirtir, modelin eğitim sürecinde farklı veri örnekleri görmesine yardımcı olur ve daha
    batch_size = 16,
    collate_fn = data_collator
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
tf_validation_set = my_model.prepare_tf_dataset(
    dataset["validation"],
    shuffle = False,
    batch_size = 16,
    collate_fn = data_collator
)

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
my_model.compile(optimizer=Adam(3e-5))

In [ ]:
my_model.fit(
    x=tf_train_set,
    validation_data = tf_validation_set,
    epochs=2
)

Epoch 1/2
533/533 [==============================] - 102s 140ms/step - loss: 0.4106 - val_loss: 0.3510
Epoch 2/2
533/533 [==============================] - 41s 76ms/step - loss: 0.2108 - val_loss: 0.3776


In [ ]:
text = "I love NLP. It's fun to analyze NLP tasks with Hugging Face"

In [ ]:
tokenized_text = tokenizer(text, return_tensors="tf")
tokenized_text

{'input_ids': <tf.Tensor: shape=(1, 19), dtype=int32, numpy=
array([[  101,  1045,  2293, 17953,  2361,  1012,  2009,  1005,  1055,
         4569,  2000, 17908, 17953,  2361,  8518,  2007, 17662,  2227,
          102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 19), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int32)>}

In [ ]:
logits = my_model(**tokenized_text).logits

In [ ]:
from tensorflow import math

In [ ]:
int(math.argmax(logits, axis=-1)[0])

1